In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
import os
import csv
import pandas as pd

### 1.1. Get the list of places

In [7]:
url2='https://www.atlasobscura.com/places?page='
sort='&sort=likes_count'

In [12]:
#DO NOT RUN THIS!
# create a file to put all the urls
f = open("urlpages.txt", "w")
for page in range(1, 401):
    url = url2 +str(page) +sort
    page_ = requests.get(url)
    soup = BeautifulSoup(page_.content, features='lxml')
    for a in soup.find_all('a', class_="content-card content-card-place"): #the class where the url is
        f.write(a.get('href')+'\n')

f.close()


### 1.2. Crawl places

In [10]:
def crawl():
    pathAncestor = os.path.join("./", "htmlpages") # Path
    #os.mkdir(pathAncestor) # create the folder in the path
    
    f = open("urlpages2.txt", "w") #create txt file with urls
    count=1728 #counting html files
    headpart='https://www.atlasobscura.com'
    
    for i in range(116, 125):
        os.makedirs(os.path.join(pathAncestor, 'page ' + str(i))) #create folder of html files
        
        url = url2 +str(i) +sort
        page_1 = requests.get(url)
        soup = BeautifulSoup(page_1.content, features='lxml')
        
        for a in soup.find_all('a', class_="content-card content-card-place"): #the class where the url is
            save=a.get('href')
            f.write(save +'\n')
            
            subdirectory = pathAncestor + "/page " + str(i) # select the corresponding folder to insert the html article
            article_name = "/article_"+str(count)+".html" 
            count+=1
            complete_path = subdirectory + article_name # insert the new complete path where create the html file
            
            with open(complete_path, "wb") as ip_file:
                link = headpart + save
                try:
                    page_2 = requests.get(link) # request the page
                except:
                    with open("failureRequest.txt", "a") as err_file: # if we loose the request place, we put into a file the link doesn't download well, then we set the "urlpages.txt" with these link
                        err_file.write(link)
                        err_file.close()

                soup = BeautifulSoup(page_2.text, features='lxml')

                ip_file.write(soup.encode('utf-8'))
                ip_file.close()

    f.close()

In [11]:
crawl()

### 1.3 Parse downloaded pages


We create the tsv files for each html file

In [2]:
def scrap(tsv_writer, article): 
    
    with open(article, 'r', encoding="utf-8") as out_file: # for each html article downloaded scrape it!
        contents = out_file.read()
        soup = BeautifulSoup(contents, features="lxml") #parse the text
        
    try:
        roi_header = soup.find_all("div", {"class": "DDPage__header-container grid-row"})[0]
        placeName = roi_header.find("h1", {"class":"DDPage__header-title"}).text.strip()

    except:
        placeName = " "
    try:
        related_list = soup.find("div",{
            "class":"card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links",
            "data-gtm-template":"DDP Footer Recirc Lists"
        })
        
        placeRelatedLists = [data.find("span").text.strip() for data in related_list.find_all("h3")]
    except:
        placeRelatedLists=''
        
    try:
        related_places = soup.find("div",{
            "class":"card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links",
            "data-gtm-template":"DDP Footer Recirc Related"
        })
        placeRelatedPlaces = [data.find("span").text.strip() for  data in related_places.find_all("h3")]
    except:
        placeRelatedPlaces=''
    try:
        placeDesc = soup.find_all("div", {"class": "DDP__body-copy"})[0]
        placeDesc = "".join([p.text.strip() for p in placeDesc.find_all("p")])
    except:
        placeDesc=''
    try:
        location = roi_header.find("div", {"class":"DDPage__header-place-location"}).text.strip()
    except:
        location=''
    try:
        counters = roi_header.find_all("div",{"class":"title-md item-action-count"})
        numPeopleVisited=int(counters[0].text.strip())
    except:
        numPeopleVisited=''
    try:
        numPeopleWant = int(counters[1].text.strip())
    except:
        numPeopleWant=''
    try:
        placeShortDesc = roi_header.find("h3", {"class":"DDPage__header-dek"}).text.strip()
    except:
        placeShortDesc=''
    try:
        sidebar = soup.find("div", {"class":"DDPageSiderail"})
        nearby_places = sidebar.find_all("div",{"class":"DDPageSiderailRecirc__item-title"})
        positions =  sidebar.find("div",{"class":"DDPageSiderail__coordinates js-copy-coordinates"}).attrs["data-coordinates"].split(",")
        placeAlt= float(positions[0].strip())
    except:
        placeAlt=''
    try:
        placeLong = float(positions[1].strip())
    except:
        placeLong=''
    try:
        placeNearby = [place.text for place in nearby_places]
    except:
        placeNearby=''
    try:
        placeAddress = [x.text for x in soup.find_all('div', {'class': 'DDPage__header-place-location'})][0]
        
    except:
        placeAddress=''
        
    try:
        footer = soup.find("div", {"id":"ugc-module"})
        Editors = footer.find_all("a",{"class":"DDPContributorsList__contributor"})
        placePubDate = footer.find("div",{"class":"DDPContributor__name"}).text
        placePubDate = datetime.strptime(placePubDate,"%B %d, %Y")
    except:
        placePubDate=''
    try:
        #10-placeEditors, 11-placePubDate, 2-placeTags
        placeEditors = [editors.find("span").text if editors.find("span") else editors.text for editors in Editors]
    except:
        placeEditors=''
    try:
        placeTags = [item.get_text("").replace("\n","") for item in soup.find_all("a",{"class":"itemTags__link js-item-tags-link"})]
        # set_trace(header="sidebar")
    except:   
        placeTags=''

    placeURL=''

    tsv_writer.writerow([placeName, placeTags, numPeopleVisited, numPeopleWant, 
                                placeDesc, placeShortDesc, placeNearby, placeAddress, placeAlt,
                                placeLong, placeEditors, placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL])


In [12]:
def create_tsv(input_path):
    path = str("./"+input_path)

    filenames = os.listdir(path)
    for i in range(116, 125):
        filenames = os.listdir(path + '/page ' + str(i))

        for file in filenames:
            with open(path + '/page ' + str(i) + '/article_'+str(file.split("_")[1].replace(".html", ""))+'.tsv', 'w', encoding="utf-8", newline='') as out_file: # create for each html article its article_i.tsv 
                tsv_writer = csv.writer(out_file, delimiter='\t')
                tsv_writer.writerow(['placeName', 'placeTags', 'numPeopleVisited', 'numPeopleWant', 
                                'placeDesc', 'placeShortDesc', 'placeNearby', 'placeAddress', 'placeAlt',
                                'placeLong', 'placeEditors', 'placePubDate','placeRelatedLists','placeRelatedPlaces','placeURL'])
                scrap(tsv_writer, path + '/page ' + str(i) + "/" + file)

In [13]:
create_tsv('htmlpages')

We create the tsv file 

In [17]:
def tsv():
    path = "./htmlpages"
    data_f = pd.DataFrame()

    for num in range(1, 401):
        filenames = os.listdir(path + '/page ' + str(num))
        for file in filenames:
            if file.endswith(".tsv"): # look for tsv files
                with open(path + '/page ' + str(num) + '/article_'+str(file.split("_")[1]), 'r', encoding="utf-8", newline='') as file_tsv:
                        df = pd.read_csv(file_tsv,sep = "\t")
                        data_f = pd.concat([data_f,df])
                            
    with open("data.tsv", "w", encoding="utf-8", newline="") as text_file: 
        text_file.write(data_f.to_csv(index=False)) # save the new file


In [18]:
tsv() #we call the tsv function to crete our dataset

In [2]:
data=pd.read_csv('/home/sophja/Documenti/ADM/HW3/data.tsv')

In [3]:
data.head()

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,Catacombes de Paris,"['ossuaries', 'memento mori', 'catacombs and c...",4457.0,7059.0,"In 2004, Parisian police were assigned to do a...","The vast, legendary catacombs hold secrets muc...","['Sculptures de Décure', 'Arago Medallions', ""...","Paris, France",48.8343,2.3322,"['CPilgrim', 'ack sed', 'ramonrodz2212', 'ging...",NaN,"['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",NaN
1,Dream House,"['optical oddities', 'eccentric homes', 'music...",766.0,6007.0,"When walking down Church Street in Tribeca, ke...","La Monte Young and Marian Zazeela's ""Dream Hou...","['Aretha Franklin Subway Tributes', 'Farm.One'...","Manhattan, New York",40.7185,-74.0048,"['seanmattison', 'erosika', 'mbison', 'Gray', ...",NaN,['30 Unexpected Places to Have a Joyful Advent...,"['Callejon de Hamel', 'Casa Neverlandia', 'Cas...",NaN
2,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...",1834.0,8608.0,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"['African Burial Ground National Monument', 'T...","Manhattan, New York",40.7134,-74.0046,"['Rebekah Otto', 'charding407', 'mbison', 'Ann...",NaN,['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",NaN
3,The Evolution Store,"['wonder cabinets', 'purveyors of curiosities'...",2199.0,6953.0,Evolution stands out among the clothing stores...,A terrific purveyor of natural history objects...,"[""Merchant's House Museum"", 'The Brown Buildin...","Manhattan, New York",40.7281,-73.9948,"['mbison', 'ramosju', 'Lauren Levesque', 'Unus...",NaN,"[""The World's Top 100 Wonders in 2018"", ""New Y...","['Kunstkammer Georg Laue', 'Ospedale delle Bam...",NaN
4,The Winchester Mystery House,"['follies and grottoes', 'outsider architectur...",3724.0,5208.0,In 1886 an eccentric woman named Sarah Winches...,A peculiar mansion built by the troubled heir ...,"['Santana Row Chess Plaza', 'Rosicrucian Park'...","San Jose, California",37.3189,-121.9506,"['mbison', 'Avoiding Regret', 'Collin', 'marle...",NaN,['12 Places to See the Work of Women Architect...,"['Casa de Piedra (Stone House)', 'Portmeirion ...",NaN


In [4]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7198 entries, 0 to 7197
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   placeName           7198 non-null   object 
 1   placeTags           7198 non-null   object 
 2   numPeopleVisited    7193 non-null   float64
 3   numPeopleWant       7193 non-null   float64
 4   placeDesc           7193 non-null   object 
 5   placeShortDesc      7193 non-null   object 
 6   placeNearby         7193 non-null   object 
 7   placeAddress        7192 non-null   object 
 8   placeAlt            7193 non-null   float64
 9   placeLong           7193 non-null   float64
 10  placeEditors        7193 non-null   object 
 11  placePubDate        0 non-null      float64
 12  placeRelatedLists   2759 non-null   object 
 13  placeRelatedPlaces  7159 non-null   object 
 14  placeURL            0 non-null      float64
dtypes: float64(6), object(9)
memory usage: 843.6+ KB
